In [20]:
import sagemaker
import pandas as pd
import numpy as np

In [3]:
# acquire endpoint
endpoint_name = 'xgboost-bikerental-v1'
predictor = sagemaker.predictor.RealTimePredictor(endpoint_name)

print ('Got Predictor ->',predictor.endpoint)

Got Predictor -> xgboost-bikerental-v1


In [4]:
from sagemaker.predictor import csv_serializer

predictor.content_type='csv'
predictor.serializer=csv_serializer

In [7]:
df_test = pd.read_csv('./data/bike_rental_test.csv')
df_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,hour,minute,second
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,0,0,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,1,0,0
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,2,0,0
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,0,0
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,4,0,0


In [13]:
x_test = df_test.iloc[:,1:].values
print (x_test.shape)

(6493, 14)


In [19]:
#predict for 10 records
x_pred=predictor.predict(x_test[:10])
print (x_pred)

b'17.67180633544922,-8.927227020263672,-13.308626174926758,0.47033411264419556,-0.46731847524642944,4.244810104370117,31.129180908203125,96.29054260253906,233.1166534423828,117.48374938964844'


In [22]:
#batch records so that we can make predictions on say 10 records at a time
predictions = []
for arr in np.array_split(x_test, 10):
    result = predictor.predict(arr)
    result = result.decode('utf-8')
    result = result.split(',')
    print (arr.shape)
    predictions += [float(r) for r in result]

(650, 14)
(650, 14)
(650, 14)
(649, 14)
(649, 14)
(649, 14)
(649, 14)
(649, 14)
(649, 14)
(649, 14)


In [23]:
print ('total predictions are ->',len(predictions))

total predictions are -> 6493


In [30]:
def adjust_counts(x):
    if (x<0):
        return 0
    else:
        return x
df_test['count'] = predictions
df_test['count'] = df_test['count'].map(adjust_counts)

In [31]:
df_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,hour,minute,second,count
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,0,0,0,17.671806
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,1,0,0,0.000000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,2,0,0,0.000000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,0,0,0.470334
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,4,0,0,0.000000


# Submit the predicted count to Kaggle !

In [32]:
df_test[['datetime','count']].to_csv('predicted_count_cloud.csv', index=False)